## testデータをpseudo-labellingするコード --
* を書くためにいろいろ検証するnotebook

In [27]:
import pandas as pd
import numpy as np

from config import *

### baselineの結果を見てみる --

In [8]:
train_df = pd.read_feather(f"{output_root}bert_baseline_lr1e-5/train_df.feather")
test_df = pd.read_feather(f"{output_root}bert_baseline_lr1e-5/test_df.feather")
test_labels = pd.read_csv(f"{output_root}bert_baseline_lr1e-5/sub_bert_baseline_lr1e-5.csv")

### 今はまだ陽性率が少ない

In [5]:
train_df["model_pred"].value_counts()

0.0    5018
1.0     238
Name: model_pred, dtype: int64

In [6]:
train_df[label_name].value_counts()

0.0    4950
1.0     306
Name: label, dtype: int64

In [12]:
test_labels["label"].value_counts()

0    3091
1     132
Name: label, dtype: int64

### 外している例の確認 --

In [14]:
train_df["annot"] = train_df["model_pred"] - train_df["label"]

In [25]:
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)
train_df[train_df["annot"] == 1].loc[:, ["source", "clean_text", "label", "model_pred"]]# model == 1, label == 0,

,source,clean_text,label,model_pred
10,livejupiter,女は精子がなくても男は卵子がなくても産めるとはおもう現段階でも倫理観が人間の進歩を疎外してるのかな,0.0,1.0
99,livejupiter,土人ばいって人間なんやで...?なんぼ教養がなく自堕落で下等ばいとしても同じ仲間なんやで...差別はあかん,0.0,1.0
130,newsplus,中国人を仲間はずれにするのはヘイトだぞ,0.0,1.0
152,news4vip,おいこら朝鮮人!なんとか言えボケ!!(*ﾟ∀ﾟ)ノ,0.0,1.0
178,livejupiter,黒人って同じ黒人同士で抜いたりするんやろかあいつら異人種過ぎて全然ムラムラしないんやけど,0.0,1.0
328,news4vip,悪だな外来種だし人殺せるしまあ全部人が悪いんだが,0.0,1.0
425,news4vip,男は田んぼで力仕事しろってのも差別になるし言い出したらきりがない,0.0,1.0
429,news4vip,わざと女の人に身体くっつける男はいるよな,0.0,1.0
496,news4vip,移民を出来るだけ入れない入れるにしても労働しない奴は即国外追放,0.0,1.0
554,newsplus,気に入らなければ俺を消せばいい朝鮮人だらけだなwwww,0.0,1.0


In [26]:
train_df[train_df["annot"] == -1].loc[:, ["source", "clean_text", "label", "model_pred"]]

,source,clean_text,label,model_pred
34,livejupiter,そら女には力はないし頭も男の方が優れてるからな女には女の仕事があって男には男の仕事がある男に満足な育児はできへんからなその点で男は女に劣ってる,1.0,0.0
46,newsplus,北朝鮮による日本人拉致と平行して在日朝鮮人の拉致も行われている日本から在日朝鮮人を拉致して工作員として訓練後再び日本へ密入国させた事件はあるよ韓国政府も新潟日赤センタービルを爆破させようと工作員を密入国させたから朝鮮人は等しく入国禁止が必要,1.0,0.0
192,newsplus,草の根で韓国に関わる人物もの全て白眼視して排斥してやるわ,1.0,0.0
207,news4vip,白人ならちょっと働くだけでアジアンの何倍もの価値を生める人生の価値が違う,1.0,0.0
291,livejupiter,デメリット有りすぎやんけ崩壊したチョン半島から日本に汚れた血がやって来るんやで,1.0,0.0
392,news4vip,消去法ならどう考えてもこのババアはないだろ!どんな選択法してんだアホ都民は,1.0,0.0
418,news4vip,女は好き嫌いや快不快で人や物事を判断する生き物ですからね,1.0,0.0
423,newsplus,イスラムに核を持たれたら終わりだから奴ら必死だよ,1.0,0.0
455,news4vip,キムチ臭ぇチョンが1匹粘着してやがるなwwったく困った有害外来種だぜww,1.0,0.0
458,news4vip,大声だして憂さ晴らししたり自分より弱そうな地元の人達を脅したりして日当3万かチョンはそんなクソ人生でも楽しいだろうななんせチョンはクソで出来ているから,1.0,0.0
